In [5]:
import catboost
from catboost import *
from catboost import datasets

In [6]:
(train_df, test_df) = catboost.datasets.amazon()

In [7]:
y = train_df.ACTION
X = train_df.drop('ACTION', axis=1)
cat_features = list(range(0, X.shape[1]))

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

In [9]:
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
valid_pool = Pool(data=X_validation, label=y_validation, cat_features=cat_features)

In [10]:
model = CatBoostClassifier(random_seed=1234,
                           iterations=10000,
                           learning_rate=0.03, 
                           depth=4, 
                           l2_leaf_reg=5,  
                           loss_function='Logloss',
                           od_type='IncToDec',
                           od_pval=0.2,
                           use_best_model=False,
                           task_type='GPU',
                           bagging_temperature=0,
                           random_strength=3,
                           one_hot_max_size=0,
                           max_ctr_complexity=4,
                           border_count=255,
                           feature_border_type='GreedyLogSum',
                           leaf_estimation_method='Gradient',
                           devices='0',
                           gpu_ram_part=0.99)
model.fit(train_pool, eval_set=valid_pool, verbose=True, plot=False)

0:	learn: 0.6556577	test: 0.6555212	best: 0.6555212 (0)	total: 40.4ms	remaining: 6m 43s
1:	learn: 0.6204961	test: 0.6202884	best: 0.6202884 (1)	total: 79.8ms	remaining: 6m 38s
2:	learn: 0.5879371	test: 0.5875462	best: 0.5875462 (2)	total: 122ms	remaining: 6m 45s
3:	learn: 0.5573324	test: 0.5565890	best: 0.5565890 (3)	total: 164ms	remaining: 6m 48s
4:	learn: 0.5289124	test: 0.5280730	best: 0.5280730 (4)	total: 206ms	remaining: 6m 51s
5:	learn: 0.5057760	test: 0.5049775	best: 0.5049775 (5)	total: 246ms	remaining: 6m 50s
6:	learn: 0.4817666	test: 0.4808030	best: 0.4808030 (6)	total: 286ms	remaining: 6m 48s
7:	learn: 0.4592907	test: 0.4572727	best: 0.4572727 (7)	total: 327ms	remaining: 6m 48s
8:	learn: 0.4387272	test: 0.4358226	best: 0.4358226 (8)	total: 366ms	remaining: 6m 45s
9:	learn: 0.4197496	test: 0.4166178	best: 0.4166178 (9)	total: 406ms	remaining: 6m 46s
10:	learn: 0.4035100	test: 0.4002608	best: 0.4002608 (10)	total: 450ms	remaining: 6m 48s
11:	learn: 0.3872011	test: 0.3833662	be

In [11]:
model.tree_count_

987

In [12]:
model.get_evals_result()['learn']['Logloss'][-1], \
model.get_evals_result()['validation']['Logloss'][-1]

(0.1482929992966813, 0.13808117314450144)

In [13]:
model.eval_metrics(train_pool, metrics=['Logloss'])['Logloss'][-1], \
model.eval_metrics(valid_pool, metrics=['Logloss'])['Logloss'][-1],

(0.05402306999217284, 0.13808129039039113)

In [14]:
from sklearn.metrics import log_loss
log_loss(train_pool.get_label(), model.predict_proba(train_pool)[:, 1]), \
log_loss(valid_pool.get_label(), model.predict_proba(valid_pool)[:, 1])

(0.0540230699921728, 0.13808129039039155)